In [18]:
import webbrowser
from bs4 import BeautifulSoup, NavigableString, Tag
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import  os
import pandas as pd
import json
import bs4
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib3 import request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


################ JUST MAKE CHANGES HERE ########################
startDate = '2021/06/01'  # <- FORMAT (YYYY/MM/DD)
endDate = '2021/06/30'
symbol = 'VTI'
################################################################

############### RESULTING FILE FORMAT ###########################

#           '<SYMBOL>_<DD>_<MM>_<YYYY>.CSV'

#################################################################

cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
graph1_df = pd.DataFrame(columns=cols)
graph2_df = pd.DataFrame(columns=cols)
file_name = ''

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d.strftime('%Y/%m/%d'))
        d += datetime.timedelta(days=1)
    return days

def main():

    global startDate
    global endDate
    global symbol
    global graph1_df
    global graph2_df
    global file_name
    cols = ['Date (DD/MM/YYYY)','Broker','Buy Value ($)','Sell Value ($)','Net Value ($)','Total Value ($)','Buy Volume','Sell Volume','Net Volume','Total Volume','Buy Trade Count','Sell Trade Count','Total Trade Count','Buy Price','Sell Price','Rank']
    

    prefs = {"profile.default_content_setting_values.notifications" : 2}  #block notifications
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument("--no-sandbox")
    # chrome_options.add_argument("--headless")     
    #chrome_options.add_argument("--disable-setuid-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    #chrome_options.add_argument('--disable-features=VizDisplayCompositor')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

    driver.get("https://scanner.daytradescans.com/signin")
    driver.implicitly_wait(20)
    driver.maximize_window()

    user_name='//*[@id="user_email"]'
    password='//*[@id="user_password"]'
    submit_button='//*[@id="new_user"]/div[2]/div/input'
    fundamentals='//*[@id="navbarDropdown"]'
    broker_data = '//*[@id="navbarSupportedContent"]/ul/li[1]/div/a[2]'
    capture = '//*[@id="capture-broker-data"]/span'
    dl = '//*[@id="canvas-link"]'

    #on_login_page
    driver.find_element_by_xpath(user_name).send_keys('ivan.cortez.paulo@gmail.com')  #login_id
    driver.find_element_by_xpath(password).send_keys("qwertyuiop2021")    #password
    driver.find_element_by_xpath(submit_button).click()
    driver.implicitly_wait(30)

    print("logged in")

    driver.find_element_by_xpath(fundamentals).click()
    driver.find_element_by_xpath(broker_data).click()

    print("in broker data")
    

    startDate = datetime.datetime.strptime(startDate, "%Y/%m/%d")
    endDate = datetime.datetime.strptime(endDate, "%Y/%m/%d")
    
    # driver.find_element_by_xpath(text_box).send_keys(input_for_search)
    # driver.find_element_by_xpath(from_date).send_keys(from_date_in)
    # driver.find_element_by_xpath(to_date).send_keys(to_date_in)
    # driver.find_element_by_xpath(search_button).click()

    import time
    
    dates = workdays(startDate,endDate)

    filedate1 = datetime.datetime.strptime(dates[0],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')
    filedate2 = datetime.datetime.strptime(dates[-1],'%Y/%m/%d').strftime('%d/%m/%Y').replace('/','_')

    for date in dates:

        new_url = f'https://scanner.daytradescans.com/brokers?code_autocomplete_label={symbol}&code=&name_autocomplete_label=&name=&from={date}&to={date}&commit=Search&minimum_share_price=&maximum_share_price=&minimum_market_cap=&maximum_market_cap=&minimum_eps=&maximum_eps=&minimum_soi=&maximum_soi='
        driver.get(new_url)
        driver.implicitly_wait(10)

        driver.find_element_by_xpath(capture).click()
        driver.find_element_by_xpath(dl).click()
        
        driver.back()
        html=driver.page_source
        soup = BeautifulSoup(html,features="lxml")
        soup_data = soup.get_text()
        a = json.loads(soup_data)

        if bool(a):
            df = pd.DataFrame(a)
            df.drop(['id'],axis=1, inplace = True)
            # import pdb; pdb.set_trace()
            df['buy_volume'] = df['buy_volume'].map(float)
            df['sell_volume'] = df['sell_volume'].map(float)

            df['total_volume'] = abs(df['buy_volume']) + abs(df['sell_volume'])
            tsum = df.tail(1)['total_volume']
            df['Rank'] = df['total_volume']
            df.Rank = df.Rank.div(float(tsum)) * 100
            df.Rank = df.Rank.round(2)

            td = datetime.datetime.strptime(date,'%Y/%m/%d').strftime('%d/%m/%Y')
            df.rename(columns={'name':'Broker','buy_value':'Buy Value ($)','sell_value':'Sell Value ($)','net_value':'Net Value ($)','total_value':'Total Value ($)','buy_volume':'Buy Volume','sell_volume':'Sell Volume','net_volume':'Net Volume','total_volume':'Total Volume','buy_trade_count':'Buy Trade Count','sell_trade_count':'Sell Trade Count','total_trade_count':'Total Trade Count','buy_price':'Buy Price','sell_price':'Sell Price'},inplace=True)
            df['Date (DD/MM/YYYY)'] = td

            df = df[cols]

            file_name = f'{symbol}_{filedate1}_to_{filedate2}.csv'
            if not(os.path.exists(file_name) and os.path.getsize(file_name) > 0):
                df.to_csv(file_name, mode = 'a',index=False)
            else:
                df.to_csv(file_name, mode = 'a', header = False, index=False)
#             import pdb; pdb.set_trace()
            graph1_df = graph1_df.append(df.tail(1))
        
    time.sleep(10)
    driver.close()


In [19]:
file_name = 'VTI_01_06_2021_to_30_06_2021.csv'

In [45]:
def graph1_plotting():
    global file_name
    
    df = pd.read_csv(file_name)
    
    df = df[df['Broker'] == 'Total']
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=('High-Low', 'Volume'), row_width=[0.2, 0.7])

    fig.add_trace(go.Candlestick(x=df['Date (DD/MM/YYYY)'], open=df['Sell Price'], high=df['Sell Price'], low=df['Buy Price'], close=df['Buy Price']),row=1, col=1)
    
    fig.add_trace(go.Bar(x=df['Date (DD/MM/YYYY)'], y=df['Total Volume'] ,showlegend=False), row=2, col=1)

    fig.update(layout_xaxis_rangeslider_visible=False)
    
    fig.show()
graph1_plotting()

In [25]:
def volume_plotting():
    global file_name
    df = pd.read_csv(file_name)
    
    broker_df = df[df['Broker'] != 'Total']
    
    fig = px.bar(broker_df, x="Date (DD/MM/YYYY)", y="Total Volume", color="Broker", hover_data=['Date (DD/MM/YYYY)'], barmode = 'stack')
    
    fig.show()

In [47]:
def overlay_plotting():
    global file_name
 
    df = pd.read_csv(file_name)
    
    broker_df = df[df['Broker'] != 'Total']
    
    fig = px.scatter(broker_df, x="Date (DD/MM/YYYY)", y=['Sell Price','Buy Price'], color="Broker", hover_data=['Date (DD/MM/YYYY)'])
    
    fig.show()
    
overlay_plotting()

In [27]:
# if __name__=="__main__":
#     main()

In [ ]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image

def graph1_plotting():
    global graph1_df
    fig = go.Figure(data=[go.Candlestick(x=graph1_df['Date (DD/MM/YYYY)'], open=graph1_df['Sell Price'], high=graph1_df['Sell Price'], low=graph1_df['Buy Price'], close=graph1_df['Buy Price'])])
    fig.show()
    
    fig.write_image("fig.png")
#     wb = Workbook()
#     sheet1 = wb.create_sheet('sheet1',0)
#     active = wb['sheet1']
#     active.add_image(Image('fig.png'),'E1')

#     wb.save('total.xlsx')
    
graph1_plotting()

In [43]:
graph1_df = pd.read_csv(file_name)
df = graph1_df[graph1_df['Broker'] != 'Total']

,Date (DD/MM/YYYY),Broker,Buy Value ($),Sell Value ($),Net Value ($),Total Value ($),Buy Volume,Sell Volume,Net Volume,Total Volume,Buy Trade Count,Sell Trade Count,Total Trade Count,Buy Price,Sell Price,Rank
0,1/6/2021,CMC Mkts,0.000,4200.000,-4.200000e+03,4200.000,0,350000,-350000,350000,0,1,1,0.000000,1.200000,43.17
1,1/6/2021,Commonwealth,4809.631,0.000,4.809631e+03,4809.631,405421,0,405421,405421,18,0,18,1.186330,0.000000,50.00
2,1/6/2021,Macquarie Insto,0.000,609.631,-6.096310e+02,609.631,0,55421,-55421,55421,0,17,17,0.000000,1.100000,6.83
3,1/6/2021,Total,4809.631,4809.631,0.000000e+00,9619.262,405421,405421,0,810842,18,18,36,1.186330,1.186330,100.00
4,2/6/2021,AIEX,48000.000,5170.000,4.283000e+04,53170.000,4000000,410000,3590000,4410000,9,2,11,1.200000,1.260976,12.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,28/06/2021,Total,40630.400,40630.400,0.000000e+00,81260.800,36689,36689,0,73378,14,14,28,110.742729,110.742729,100.00
162,29/06/2021,Unassigned Transactions,29232.455,29232.455,-3.637980e-12,58464.910,25105,25105,0,50210,33,33,66,116.440769,116.440769,100.00
163,29/06/2021,Total,29232.455,29232.455,0.000000e+00,58464.910,25105,25105,0,50210,33,33,66,116.440769,116.440769,100.00
164,30/06/2021,Unassigned Transactions,52483.255,52483.255,-7.275960e-12,104966.510,43309,43309,0,86618,34,34,68,121.183253,121.183253,100.00


In [74]:
lob = df['Broker'].unique().tolist()
df1 = df[df['Broker'] == lob[0]]
df2 = df[df['Broker'] == lob[1]]
df3 = df[df['Broker'] == lob[2]]
df4 = df[df['Broker'] == lob[3]]
df5 = df[df['Broker'] == lob[4]]

In [80]:

    
fig = make_subplots(rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.03, subplot_titles=('High-Low',''), row_width=[0.2, 0.7])

fig.add_trace(go.Candlestick(x=df1['Date (DD/MM/YYYY)'], open=df1['Sell Price'], high=df1['Sell Price'], low=df1['Buy Price'], close=df1['Buy Price'], name=lob[0]),row=1, col=1)
fig.add_trace(go.Candlestick(x=df2['Date (DD/MM/YYYY)'], open=df2['Sell Price'], high=df2['Sell Price'], low=df2['Buy Price'], close=df2['Buy Price'], name=lob[1]),row=1, col=1)
fig.add_trace(go.Candlestick(x=df3['Date (DD/MM/YYYY)'], open=df3['Sell Price'], high=df3['Sell Price'], low=df3['Buy Price'], close=df3['Buy Price'], name=lob[2]),row=1, col=1)
fig.add_trace(go.Candlestick(x=df4['Date (DD/MM/YYYY)'], open=df4['Sell Price'], high=df4['Sell Price'], low=df4['Buy Price'], close=df4['Buy Price'], name=lob[3]),row=1, col=1)
fig.add_trace(go.Candlestick(x=df5['Date (DD/MM/YYYY)'], open=df5['Sell Price'], high=df5['Sell Price'], low=df5['Buy Price'], close=df5['Buy Price'], name=lob[4]),row=1, col=1)

fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

In [83]:
df3

,Date (DD/MM/YYYY),Broker,Buy Value ($),Sell Value ($),Net Value ($),Total Value ($),Buy Volume,Sell Volume,Net Volume,Total Volume,Buy Trade Count,Sell Trade Count,Total Trade Count,Buy Price,Sell Price,Rank
2,1/6/2021,Macquarie Insto,0.0,609.631,-609.631,609.631,0,55421,-55421,55421,0,17,17,0.0,1.1,6.83
112,18/06/2021,Macquarie Insto,0.0,9485.200,-9485.200,9485.200,0,8248,-8248,8248,0,2,2,0.0,115.0,16.97
